In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import sys
import math

# local code
sys.path.insert(1, "./code/")
from Utils import Utils # student's library
from Eval import Eval # student's library
from Extract import Extract # student's library


In [2]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")

In [3]:
# sanity checks
#train_df.id.nunique()
# 19579

#train_df.author.unique()
# array(['EAP', 'HPL', 'MWS'], dtype=object)

In [4]:
Utils.check_for_nulls(train_df)

{'author': 0, 'id': 0, 'text': 0}

In [5]:
short_df = train_df[:20]
short_df.head(1)

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP


In [31]:
import spacy
spacy_model = spacy.load('en_core_web_sm')

In [7]:
# short_sents = [tokenize(x) for x in list(short_df.text)]
# short_sents[0]

In [8]:
import numpy as np
import gensim

# http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/

# this glove code is wrong type for later work
# with open("data/glove.42B.300d.txt", "rb") as lines:
#     w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
#            for line in lines}


# let X be a list of tokenized texts (i.e. list of lists of tokens)
# sentences param is token lists

# model = gensim.models.Word2Vec(short_sents, size=100)
# w2v = dict(zip(model.wv.index2word, model.wv.vectors))
# type(w2v)

In [9]:
#list(w2v.keys())[9990:9999]
# [b'iconic',
#  b'erp',
#  b'crest',
#  b'radius',
#  b'spiral',
#  b'nyse',
#  b'lotion',
#  b'oriental',
#  b'admire']

In [10]:
# a_key = list(w2v.keys())[9999]
# a_key.decode("utf-8")

In [11]:
#type(w2v[str.encode('owl')])

In [12]:
#https://radimrehurek.com/gensim/models/doc2vec.html
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#TaggedDocument does not filter or stem

# documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(short_sents)]
# model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

# vector = model.infer_vector(["system", "response"])
# vector

In [13]:
# this is it! just use the output of infer_vector in training model

In [14]:
# fast bad data data

# regular data
#     train: 19580 * .9 rows
#     test:  8393 rows
#     val:   19580 * .1 rows
  
VAL_IDX =  math.ceil(len(train_df) * .8)
TEST_IDX =  math.ceil(len(train_df) * .9)
traindata = train_df[:VAL_IDX]
valdata = train_df[VAL_IDX:TEST_IDX]
testdata = train_df[TEST_IDX:]

print(VAL_IDX, TEST_IDX)

15664 17622


In [15]:
# example_tagged_doc = TaggedDocument(short_sents[0], [0])
# example_tagged_doc

In [27]:
# order of ops:
# just grammar features
# just tfidf vectorizer
# both

# kmeans

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(traindata.text)
Y_train = traindata.author #short_df.author

X_test = vectorizer.transform(testdata.text) #short_df.text
Y_test = list(testdata.author)

In [28]:
# linear svc
from sklearn.svm import LinearSVC

lin_clf = LinearSVC()
lin_clf.fit(X_train, Y_train) 

preds = lin_clf.predict(X_test)
preds

array(['EAP', 'MWS', 'MWS', ..., 'EAP', 'MWS', 'HPL'], dtype=object)

In [29]:
# evaluation

accuracy = Eval.get_accuracy(preds, Y_test)
print("Accuracy: ", accuracy)

<class 'numpy.ndarray'> <class 'list'>
Accuracy:  0.8421052631578947


In [30]:
X_train[0]

<1x23090 sparse matrix of type '<class 'numpy.float64'>'
	with 34 stored elements in Compressed Sparse Row format>